In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklego.mixture import GMMOutlierDetector
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.metrics.pairwise import polynomial_kernel

In [ ]:
celeba_features = np.load("celeba_features.npy")

In [ ]:
celeba_features.shape

In [ ]:
ss = StandardScaler()
ss.fit(celeba_features)
celeba_features = ss.transform(celeba_features)

In [ ]:
pca = PCA(n_components=512, whiten=True)
pca = pca.fit(celeba_features)
print('Explained variance percentage = %0.2f' % sum(pca.explained_variance_ratio_))
celeba_features = pca.transform(celeba_features)

In [ ]:
from sklearn.kernel_approximation import PolynomialCountSketch

In [ ]:
feature_map_poly = PolynomialCountSketch(gamma=.2, random_state=1)

In [ ]:
fmp = feature_map_poly.fit(celeba_features)
celeba_features = feature_map_poly.transform(celeba_features)

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
clf = EllipticEnvelope(random_state=0).fit(celeba_features)

In [ ]:
attr_root = "results_without_sharpen_features.txt"

In [ ]:
attr_root_extra = "results_with_sharpen_labels.txt"

In [ ]:
file_to_write_sharpen_low_confidence = open("labels_filtered_train.txt","a")
c = 0
yfcc1m_features = open(attr_root, "r")
yfcc1m_labels = open(attr_root_extra, "r").readlines()

for i, aline in enumerate(tqdm(yfcc1m_features, total=len(yfcc1m_labels))):
    values = aline.split()
    image = values[0]
    features = np.array(values[1:]).astype(float)
    labels = np.array(yfcc1m_labels[i].split()[1:]).astype(float)
    #print(clf.predict(fmp.transform(pca.transform(ss.transform(features.reshape(1,-1))))))
    if clf.predict(fmp.transform(pca.transform(ss.transform(features.reshape(1,-1))))) == 1:
        file_to_write_sharpen_low_confidence.write(image+'\t'+'\t'.join(list(labels.astype(str))) +"\n")
        c += 1

In [ ]:
yfcc1m_features.close()